In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [3]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [4]:
# The number of steps within one time segment
TIME_PERIODS = 1500
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 1500
N_FEATURES = 8

In [5]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'Class_label']


In [6]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [7]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [8]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [9]:
 pd.options.display.float_format = "{:,.5f}".format

In [10]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [11]:
verbose, epochs, batch_size = 0, 120, 16

In [12]:
# def step_decay(epoch):
#    initial_lrate = 0.1
#    drop = 0.1
#    epochs_drop = 16.0
#    lrate = initial_lrate * tf.math.pow(drop,  
#            tf.math.floor((1+epoch)/epochs_drop))
#    return lrate

In [13]:
def step_decay(epoch):
   initial_lrate = 1e-3
   drop = 0.1
   epochs_drop = 40.0
   lrate = initial_lrate * tf.math.pow(drop,  
           tf.math.floor((1+epoch)/epochs_drop))
   return lrate

In [14]:
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)

In [15]:
# lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]

In [16]:
# import os
path='/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/olsson/sub_wise_process_TT/'

In [17]:
for i in range(1,37):
    file_path_train=path+'S'+str(i)+'_'+'tr'+'.'+'csv'
    print(file_path_train)
    df_Train=read_data_Train(file_path_train)
    show_basic_dataframe_info(df_Train)
    df_Train.head(5)
    file_path_test=path+'S'+str(i)+'_'+'tt'+'.'+'csv'
    print(file_path_test)
    df_Test=read_data_Test(file_path_test)
    show_basic_dataframe_info(df_Test)
    df_Test.head(5)
    
    scaler = preprocessing.StandardScaler().fit(df_Train.iloc[:,0:N_FEATURES])
    df_Train.iloc[:,0:N_FEATURES]=scaler.transform(df_Train.iloc[:,0:N_FEATURES])
    df_Test.iloc[:,0:N_FEATURES]=scaler.transform(df_Test.iloc[:,0:N_FEATURES])
       
    LABEL = 'ActivityEncoded'
    # Transform the labels from String to Integer via LabelEncoder
    le = preprocessing.LabelEncoder()
    # Add a new column to the existing DataFrame with the encoded values
    df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
    # df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
    df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())
    x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_train shape: ', x_train.shape)
    # print(x_train)
    print(x_train.shape[0], 'training samples')
    print('y_train shape: ', y_train.shape)
    # Set input & output dimensions
    num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
    print('num_time_periods',num_time_periods)
    print('num_sensors',num_sensors)
    num_classes = le.classes_.size
    print('class_list',list(le.classes_))
    # input_shape = (num_time_periods,num_sensors)
    # print(input_shape)
    input_shape = (num_time_periods,num_sensors)
    #x_train = x_train.reshape(x_train.shape[0], input_shape)
    print('x_train shape:', x_train[0].shape)
    print('input_shape:', input_shape)
    x_train = x_train.astype('float32')
    # x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
    # y_train = y_train.astype('float32')
    print('y_train shape',y_train.shape)
    y_train_hot = np_utils.to_categorical(y_train, num_classes)
#     print(y_train_hot)
    # y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
    print('New y_train shape: ', y_train_hot.shape)
    x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_test shape: ', x_test.shape)
    # print(x_train)
    print(x_test.shape[0], 'testing samples')
    print('y_test shape: ', y_test.shape)
    # Set input_shape / reshape for Keras
    #x_test = x_test.reshape(x_test.shape[0], input_shape)
    x_test = x_test.astype('float32')
    y_test = y_test.astype('float32')
    y_test_hot = np_utils.to_categorical(y_test, num_classes)
    # n_steps, n_length = 20, 25
    # n_steps, n_length= 10, 50
    # n_steps, n_length= 16, 32
    n_steps, n_length = 6, 250
    n_depth=8
    x_train = x_train.reshape(x_train.shape[0], n_steps, n_length,n_depth)
    print('x_train shape: ', x_train.shape)
    # x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
    # print('x_valid shape: ', x_valid.shape)
    x_test = x_test.reshape(x_test.shape[0], n_steps, n_length,n_depth)
    print('x_test shape: ', x_test.shape)
    n_outputs = y_train_hot.shape[1]
    print('n_outputs',n_outputs)
   # chen architecture
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=32, kernel_size=2, padding='same', kernel_initializer="he_normal",strides=1,kernel_regularizer=l1(1e-04)), \
                               input_shape=(n_steps,n_length,n_depth)))
    model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=4,strides=2)))
    model.add(TimeDistributed(Activation('tanh')))
    model.add(TimeDistributed(Conv1D(filters=16, kernel_size=2,padding="same",kernel_initializer="he_normal",strides=1,kernel_regularizer=l1(1e-04))))
    model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=4,strides=2)))
    model.add(TimeDistributed(Activation('tanh')))
    model.add(TimeDistributed(Dropout(0.2093)))
    model.add(TimeDistributed(Activation('relu')))
    model.add(TimeDistributed(Flatten()))
    # model.add(Flatten())
    model.add(LSTM(64,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(32,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='tanh'))
    model.add(BatchNormalization(epsilon=1e-05, momentum=0.9, weights=None))
    model.add(Dense(n_outputs, activation='softmax'))
    adam=optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999,epsilon=1e-08, amsgrad=False)
    checkpoint_filepath = '/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/CNN25X20/checkpoint.hdf5'
    # model.load_weights(checkpoint_filepath) 
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
    early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
    print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)
    csv_logger = CSVLogger('/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/CNN25X20/LSTM_nina_20X100.csv', append=True, separator=';')
    history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,lrate,early],validation_data=(x_test, y_test_hot), verbose=1)
    best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
    print('epoch_number',best_index+1)
    print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])
    model.load_weights(checkpoint_filepath) 
    _, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
    print('test_accuracy',testaccuracy)
    test_acc.append(testaccuracy)
    print(test_acc)
    print('test_mean for %d subjects:',i)
    test_mean=statistics.mean(test_acc)
    print(test_mean)

/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/olsson/sub_wise_process_TT/S1_tr.csv
Number of columns in the dataframe: 9
Number of rows in the dataframe: 27000

/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/olsson/sub_wise_process_TT/S1_tt.csv
Number of columns in the dataframe: 9
Number of rows in the dataframe: 9000

x_train shape:  (17, 1500, 8)
17 training samples
y_train shape:  (17,)
num_time_periods 1500
num_sensors 8
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]
x_train shape: (1500, 8)
input_shape: (1500, 8)
y_train shape (17,)
New y_train shape:  (17, 6)
x_test shape:  (5, 1500, 8)
5 testing samples
y_test shape:  (5,)
x_train shape:  (17, 6, 250, 8)
x_test shape:  (5, 6, 250, 8)
n_outputs 6
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 6, 250, 32)        544       
_____________________________________________________

2/2 [==============================] - 0s 59ms/step - loss: 0.1253 - accuracy: 0.9608 - val_loss: 2.2942 - val_accuracy: 0.6000

Epoch 00022: val_accuracy did not improve from 0.80000
Epoch 23/120
2/2 [==============================] - 0s 59ms/step - loss: 0.1254 - accuracy: 0.9608 - val_loss: 2.4934 - val_accuracy: 0.6000

Epoch 00023: val_accuracy did not improve from 0.80000
Epoch 24/120
2/2 [==============================] - 0s 52ms/step - loss: 0.1369 - accuracy: 1.0000 - val_loss: 2.1837 - val_accuracy: 0.6000

Epoch 00024: val_accuracy did not improve from 0.80000
Epoch 25/120
2/2 [==============================] - 0s 55ms/step - loss: 0.1120 - accuracy: 0.9608 - val_loss: 2.2401 - val_accuracy: 0.6000

Epoch 00025: val_accuracy did not improve from 0.80000
Epoch 26/120
2/2 [==============================] - 0s 57ms/step - loss: 0.1074 - accuracy: 0.9608 - val_loss: 2.3955 - val_accuracy: 0.6000

Epoch 00026: val_accuracy did not improve from 0.80000
Epoch 27/120
2/2 [==========


Epoch 00063: val_accuracy did not improve from 0.80000
Epoch 64/120
2/2 [==============================] - 0s 55ms/step - loss: 0.1019 - accuracy: 0.9608 - val_loss: 1.8991 - val_accuracy: 0.6000

Epoch 00064: val_accuracy did not improve from 0.80000
Epoch 65/120
2/2 [==============================] - 0s 57ms/step - loss: 0.1005 - accuracy: 1.0000 - val_loss: 1.8781 - val_accuracy: 0.6000

Epoch 00065: val_accuracy did not improve from 0.80000
Epoch 66/120
2/2 [==============================] - 0s 52ms/step - loss: 0.1050 - accuracy: 0.9608 - val_loss: 1.7464 - val_accuracy: 0.4000

Epoch 00066: val_accuracy did not improve from 0.80000
Epoch 67/120
2/2 [==============================] - 0s 57ms/step - loss: 0.1060 - accuracy: 0.9608 - val_loss: 1.6958 - val_accuracy: 0.4000

Epoch 00067: val_accuracy did not improve from 0.80000
Epoch 68/120
2/2 [==============================] - 0s 50ms/step - loss: 0.1064 - accuracy: 0.9608 - val_loss: 2.0218 - val_accuracy: 0.6000

Epoch 00068: v

2/2 [==============================] - 0s 81ms/step - loss: 0.1064 - accuracy: 0.9608 - val_loss: 1.9662 - val_accuracy: 0.6000

Epoch 00105: val_accuracy did not improve from 0.80000
Epoch 106/120
2/2 [==============================] - 0s 69ms/step - loss: 0.1048 - accuracy: 0.9608 - val_loss: 2.0378 - val_accuracy: 0.6000

Epoch 00106: val_accuracy did not improve from 0.80000
Epoch 107/120
2/2 [==============================] - 0s 69ms/step - loss: 0.1067 - accuracy: 0.9608 - val_loss: 2.3140 - val_accuracy: 0.6000

Epoch 00107: val_accuracy did not improve from 0.80000
Epoch 108/120
2/2 [==============================] - 0s 65ms/step - loss: 0.1057 - accuracy: 0.9608 - val_loss: 2.4937 - val_accuracy: 0.6000

Epoch 00108: val_accuracy did not improve from 0.80000
Epoch 109/120
2/2 [==============================] - 0s 62ms/step - loss: 0.1045 - accuracy: 0.9608 - val_loss: 2.4710 - val_accuracy: 0.6000

Epoch 00109: val_accuracy did not improve from 0.80000
Epoch 110/120
2/2 [=====

2/2 [==============================] - 0s 55ms/step - loss: 0.2391 - accuracy: 0.9608 - val_loss: 1.3364 - val_accuracy: 0.4000

Epoch 00011: val_accuracy did not improve from 0.60000
Epoch 12/120
2/2 [==============================] - 0s 58ms/step - loss: 0.2999 - accuracy: 0.8407 - val_loss: 1.2671 - val_accuracy: 0.6000

Epoch 00012: val_accuracy did not improve from 0.60000
Epoch 13/120
2/2 [==============================] - 0s 68ms/step - loss: 0.2495 - accuracy: 0.9608 - val_loss: 1.0981 - val_accuracy: 0.6000

Epoch 00013: val_accuracy did not improve from 0.60000
Epoch 14/120
2/2 [==============================] - 0s 60ms/step - loss: 0.2593 - accuracy: 0.9007 - val_loss: 0.9775 - val_accuracy: 0.8000

Epoch 00014: val_accuracy improved from 0.60000 to 0.80000, saving model to /media/naveen/nav/mat_codes/EMG_data_for_gestures-master/CNN25X20/checkpoint.hdf5
Epoch 15/120
2/2 [==============================] - 0s 64ms/step - loss: 0.2099 - accuracy: 0.9608 - val_loss: 0.7981 - va

...

2/2 [==============================] - 0s 55ms/step - loss: 0.1088 - accuracy: 0.9608 - val_loss: 0.6312 - val_accuracy: 0.8000

Epoch 00082: val_accuracy did not improve from 1.00000
Epoch 83/120
2/2 [==============================] - 0s 56ms/step - loss: 0.1019 - accuracy: 0.9608 - val_loss: 0.6403 - val_accuracy: 0.8000

Epoch 00083: val_accuracy did not improve from 1.00000
Epoch 84/120
2/2 [==============================] - 0s 53ms/step - loss: 0.1002 - accuracy: 0.9608 - val_loss: 0.5556 - val_accuracy: 0.8000

Epoch 00084: val_accuracy did not improve from 1.00000
Epoch 85/120
2/2 [==============================] - 0s 52ms/step - loss: 0.1076 - accuracy: 0.9608 - val_loss: 0.3136 - val_accuracy: 0.8000

Epoch 00085: val_accuracy did not improve from 1.00000
Epoch 86/120
2/2 [==============================] - 0s 50ms/step - loss: 0.1000 - accuracy: 1.0000 - val_loss: 0.5108 - val_accuracy: 0.8000

Epoch 00086: val_accuracy did not improve from 1.00000
Epoch 87/120
2/2 [==========

y_train shape (17,)
New y_train shape:  (17, 6)
x_test shape:  (5, 1500, 8)
5 testing samples
y_test shape:  (5,)
x_train shape:  (17, 6, 250, 8)
x_test shape:  (5, 6, 250, 8)
n_outputs 6
Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_385 (TimeDi (None, 6, 250, 32)        544       
_________________________________________________________________
time_distributed_386 (TimeDi (None, 6, 250, 32)        128       
_________________________________________________________________
time_distributed_387 (TimeDi (None, 6, 124, 32)        0         
_________________________________________________________________
time_distributed_388 (TimeDi (None, 6, 124, 32)        0         
_________________________________________________________________
time_distributed_389 (TimeDi (None, 6, 124, 16)        1040      
_______________________________________________________________

2/2 [==============================] - 0s 202ms/step - loss: 0.1056 - accuracy: 0.9608 - val_loss: 0.1747 - val_accuracy: 1.0000

Epoch 00025: val_accuracy did not improve from 1.00000
Epoch 26/120
2/2 [==============================] - 0s 58ms/step - loss: 0.1182 - accuracy: 0.9608 - val_loss: 0.0957 - val_accuracy: 1.0000

Epoch 00026: val_accuracy did not improve from 1.00000
Epoch 27/120
2/2 [==============================] - 0s 48ms/step - loss: 0.1230 - accuracy: 0.9608 - val_loss: 0.1227 - val_accuracy: 1.0000

Epoch 00027: val_accuracy did not improve from 1.00000
Epoch 28/120
2/2 [==============================] - 0s 56ms/step - loss: 0.1121 - accuracy: 0.9608 - val_loss: 0.1159 - val_accuracy: 1.0000

Epoch 00028: val_accuracy did not improve from 1.00000
Epoch 29/120
2/2 [==============================] - 0s 59ms/step - loss: 0.1054 - accuracy: 0.9608 - val_loss: 0.2024 - val_accuracy: 1.0000

Epoch 00029: val_accuracy did not improve from 1.00000
Epoch 30/120
2/2 [=========


Epoch 00066: val_accuracy did not improve from 1.00000
Epoch 67/120
2/2 [==============================] - 0s 55ms/step - loss: 0.1049 - accuracy: 0.9608 - val_loss: 0.1781 - val_accuracy: 1.0000

Epoch 00067: val_accuracy did not improve from 1.00000
Epoch 68/120
2/2 [==============================] - 0s 53ms/step - loss: 0.1518 - accuracy: 0.9007 - val_loss: 0.1557 - val_accuracy: 1.0000

Epoch 00068: val_accuracy did not improve from 1.00000
Epoch 69/120
2/2 [==============================] - 0s 55ms/step - loss: 0.1115 - accuracy: 0.9608 - val_loss: 0.3105 - val_accuracy: 0.8000

Epoch 00069: val_accuracy did not improve from 1.00000
Epoch 70/120
2/2 [==============================] - 0s 58ms/step - loss: 0.1077 - accuracy: 0.9608 - val_loss: 0.1835 - val_accuracy: 0.8000

Epoch 00070: val_accuracy did not improve from 1.00000
Epoch 71/120
2/2 [==============================] - 0s 59ms/step - loss: 0.0975 - accuracy: 1.0000 - val_loss: 0.0632 - val_accuracy: 1.0000

Epoch 00071: v

2/2 [==============================] - 0s 59ms/step - loss: 0.1069 - accuracy: 0.9608 - val_loss: 0.0474 - val_accuracy: 1.0000

Epoch 00108: val_accuracy did not improve from 1.00000
Epoch 109/120
2/2 [==============================] - 0s 53ms/step - loss: 0.1071 - accuracy: 0.9608 - val_loss: 0.0436 - val_accuracy: 1.0000

Epoch 00109: val_accuracy did not improve from 1.00000
Epoch 110/120
2/2 [==============================] - 0s 55ms/step - loss: 0.1021 - accuracy: 0.9608 - val_loss: 0.0507 - val_accuracy: 1.0000

Epoch 00110: val_accuracy did not improve from 1.00000
Epoch 111/120
2/2 [==============================] - 0s 65ms/step - loss: 0.0988 - accuracy: 1.0000 - val_loss: 0.0467 - val_accuracy: 1.0000

Epoch 00111: val_accuracy did not improve from 1.00000
Epoch 112/120
2/2 [==============================] - 0s 58ms/step - loss: 0.1113 - accuracy: 0.9608 - val_loss: 0.0502 - val_accuracy: 1.0000

Epoch 00112: val_accuracy did not improve from 1.00000
Epoch 113/120
2/2 [=====

In [18]:
# statistics.mean(test_acc)
test_std=statistics.stdev(test_acc)
print(test_std)

0.12175957891710966
